In [ ]:
#Importing all the values
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPool2D,BatchNormalization, LSTM, CuDNNLSTM, MaxPooling2D
from keras.models import Sequential
from keras import regularizers
import keras
from keras.layers import TimeDistributed
from keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers , models, optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

In [ ]:

#Initializing the funtion to plot the accuracy plot along with the loss plot
import os
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import numpy as np
from tqdm import notebook
import matplotlib.pyplot as plt


def plot_map(history):
    
    #Training losss and validation loss plot
    plt.plot( list(range(1, 1+len(history.history['loss']))), history.history['loss'])
    plt.plot(list(range(1, 1+len(history.history['val_loss']))), history.history['val_loss'])
    plt.xlabel("Epochs")
    plt.legend(['Training loss', 'Validation loss'])
    plt.show()
    
    #Accuracy plots
    plt.plot( list(range(1, 1+len(history.history['accuracy']))), history.history['accuracy'])
    plt.plot(list(range(1, 1+len(history.history['val_accuracy']))), history.history['val_accuracy'])
    plt.xlabel("Epochs")
    plt.legend(['Training accuracy', 'Validation accuracy'])
    plt.show()
    



##This is for the data upload

In [ ]:
datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.3)

#Importing the images from the directory

#This is for the train dataset
train_generator = datagen.flow_from_directory(
        '../input/fer2013/train',
        target_size=(80, 80),
        batch_size=64,
        color_mode="rgb",
        class_mode='categorical',
        subset = 'training'
)



#This is for the validation dataset
validation_generator = datagen.flow_from_directory(
        '../input/fer2013/train',
        target_size=(80, 80),
        batch_size=64,
        color_mode="rgb",
        class_mode='categorical',
        subset = 'validation'
)

#Test datagenerator
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        '../input/fer2013/test',
        target_size=(80, 80),
        batch_size=64,
        color_mode="rgb",
        class_mode='categorical',
)

##Base Model

In [ ]:


#This is out first base line trial, in order to check for the Inception net model
base_model = keras.applications.inception_v3.InceptionV3(input_shape = (80, 80, 3),include_top = False, weights = 'imagenet')


#No trainable parameters
for i in base_model.layers:
    i.trainable = False

#Adding the top layers
output_model = base_model.output
output_model = Flatten()(output_model)
output_model = BatchNormalization()(output_model)
output_model = Dense(512 , activation = "relu")(output_model)
output_model = BatchNormalization()(output_model)
output_model = Dense(128 , activation = "relu")(output_model)
output_model = BatchNormalization()(output_model)
output_model = Dense(7 , activation  = "softmax")(output_model)

model_ite1 = Model(inputs = base_model.input, outputs = output_model)

model_ite1.compile(tf.keras.optimizers.Adam(), loss='CategoricalCrossentropy', metrics = ['accuracy'])

#Plotting the map
history_iter1 = model_ite1.fit(train_generator, epochs = 10, validation_data = validation_generator)
plot_map(history_iter1)

In [ ]:
#Testing values
y_pred = model_ite1.predict(test_generator)

#Printing out the classification report
from sklearn.metrics import classification_report
print(classification_report(test_generator.classes, np.argmax(y_pred, axis=1)))

In [ ]:
#These are the classes
print(train_generator.class_indices)

In graph we are able to see that validation loss increases tremendously while training still keeps of decreasing which shows that our model has started to remember the inputs => OVERFITTING
Moreover on our testing data accuracy = 0.17 (17%) which also shows that our model is not able to predict newly shown dataset. F1 scores shows that disgust and surprise are the datasets which have the worst prediction rate.

In [ ]:
#To check which should be our loss function, we ran a grid search on the
#loss function to decide between them

In [ ]:
#With this we can get the values of x_train and y_train and validation values, from the generators
x_train, y_train = next(train_generator)
x_val, y_val = next(validation_generator)

#Converting the one hot encoded values to normal values
y_train_sparse = np.argmax(y_train, axis = 1)
y_val_sparse = np.argmax(y_val, axis = 1)

In [ ]:

#This is the function for the grid search 
def model_built(activation):
    print("activation: ", activation)
    base_model = keras.applications.inception_v3.InceptionV3(input_shape = (80, 80, 3),include_top = False, weights = 'imagenet')
    
    #No trainable layers
    for i in base_model.layers:
        i.trainable = False
    
    #Top layers
    output_model = base_model.output
    output_model = Flatten()(output_model)
    output_model = BatchNormalization()(output_model)
    output_model = Dense(512 , activation = activation)(output_model)
    output_model = BatchNormalization()(output_model)
    output_model = Dense(128 , activation = activation)(output_model)
    output_model = BatchNormalization()(output_model)
    output_model = Dense(7 , activation  = "softmax")(output_model)
    
    model_ite1 = Model(inputs = base_model.input, outputs = output_model)
    #Optimizer
    model_ite1.compile(tf.keras.optimizers.Adam(), loss='SparseCategoricalCrossentropy', metrics = ['accuracy'])
    print("Model Compiled")
    return model_ite1

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adagrad, Adam

#Activation is being checked in the grid search
param_grid = {
    'activation': ['tanh', 'relu'],
}

#Doing grid search and printing out values
grid_search = GridSearchCV(estimator=KerasClassifier(model_built), param_grid=param_grid, scoring='accuracy', cv=3)
grid_search.fit(x_train, y_train_sparse, epochs=2)
print("Best parameters: ", grid_search.best_params_)

In [ ]:
#These are the final results that we have got after doing the grid search on the parameters

import pandas as pd
print(pd.DataFrame(grid_search.cv_results_))

In [ ]:
#Grid search on the drop out layers

def model_built(dropout):
    print("dropout: ", dropout)
    base_model_2 = keras.applications.inception_v3.InceptionV3(input_shape = (80,80,3),include_top = False, weights = 'imagenet')

    for i in base_model_2.layers:
        i.trainable = False

    output_model = base_model_2.output
    output_model = Flatten()(output_model)
    output_model = BatchNormalization()(output_model)
    ouput_model = Dropout(dropout)(output_model)
    output_model = Dense(512 , activation = "tanh")(output_model)
    output_model = BatchNormalization()(output_model)
    #CHecking how many to be dropped
    ouput_model = Dropout(dropout)(output_model)
    output_model = Dense(128 , activation = "tanh")(output_model)
    output_model = BatchNormalization()(output_model)
    output_model = Dense(7 , activation  = "softmax")(output_model)

    model_ite2 = Model(inputs = base_model_2.input, outputs = output_model)

    model_ite2.compile(tf.keras.optimizers.Adam(), loss='SparseCategoricalCrossentropy', metrics = ['accuracy'])
    print("Model Compiled")
    return model_ite2

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adagrad, Adam

#Grid search on the dropout layers
param_grid = {
    'dropout': [0.2, 0.25, 0.5],
}

grid_search = GridSearchCV(estimator=KerasClassifier(model_built), param_grid=param_grid, scoring='accuracy', cv=3)
#Printing out the final values
grid_search.fit(x_train, y_train_sparse, epochs=2)
print("Best parameters: ", grid_search.best_params_)

In [ ]:
#Rinning model based on grid search

base_model_2 = keras.applications.inception_v3.InceptionV3(input_shape = (80,80,3),include_top = False, weights = 'imagenet')
#NO trainable layers
for i in base_model_2.layers:
    i.trainable = False

output_model = base_model_2.output
output_model = Flatten()(output_model)
output_model = BatchNormalization()(output_model)
ouput_model = Dropout(0.5)(output_model)
output_model = Dense(512 , activation = "tanh")(output_model)
output_model = BatchNormalization()(output_model)
output_model = Dense(128 , activation = "tanh")(output_model)
output_model = BatchNormalization()(output_model)
output_model = Dense(7 , activation  = "softmax")(output_model)

model_ite2 = Model(inputs = base_model_2.input, outputs = output_model)

model_ite2.compile(tf.keras.optimizers.Adam(), loss='CategoricalCrossentropy', metrics = ['accuracy'])

history_iter2 = model_ite2.fit(train_generator, epochs = 10, validation_data = validation_generator)
plot_map(history_iter2)

In [ ]:
#Now we are training the model with more trainable layers and unfreezing the 
#parameters of the inception net model
base_model_4 = keras.applications.inception_v3.InceptionV3(input_shape = (80,80,3),include_top = False, weights = 'imagenet')

for i in base_model_4.layers[:-15]:
    i.trainable = False

output_model = base_model_4.output
output_model = Flatten()(output_model)
output_model = BatchNormalization()(output_model)
ouput_model = Dropout(0.2)(output_model)
output_model = Dense(512 , activation = "relu", kernel_regularizer=regularizers.l2(0.006))(output_model)
output_model = BatchNormalization()(output_model)
ouput_model = Dropout(0.2)(output_model)
output_model = Dense(128 , activation = "relu", kernel_regularizer=regularizers.l2(0.006))(output_model)
output_model = BatchNormalization()(output_model)
output_model = Dense(7 , activation  = "softmax")(output_model)

model_ite4 = Model(inputs = base_model_4.input, outputs = output_model)

model_ite4.compile(tf.keras.optimizers.Adam(), loss='CategoricalCrossentropy', metrics = ['accuracy'])

history_iter4 = model_ite4.fit(train_generator, epochs = 10, validation_data = validation_generator)
plot_map(history_iter4)

In [ ]:
#Predicting the output on the test dataset
y_pred = model_ite4.predict(test_generator)

from sklearn.metrics import classification_report
print(classification_report(test_generator.classes, np.argmax(y_pred, axis=1)))

It is visible that the training accuracy gets stagnant at the accuracy of 0.41 - 0.44

Let's try data augmentation : Another method to overcome overfitting

In [ ]:
from sklearn.utils import compute_class_weight

#Image size to 80x80 along with more augmentations
img_shape = 80
batch_size = 80
train_data_path = '../input/fer2013/train/'
test_data_path = '../input/fer2013/test/'

#For training dataset
train_preprocessor = ImageDataGenerator(
        rescale = 1 / 255.,
        rotation_range=5,
        width_shift_range=0.2, 
        height_shift_range=0.2,
        zoom_range = 0.2,
        horizontal_flip=True
    
    )


#For testing dataset
test_preprocessor = ImageDataGenerator(
    rescale = 1 / 255.,
    rotation_range=5,
    width_shift_range=0.2, 
    height_shift_range=0.2,
    zoom_range = 0.2,
    horizontal_flip=True
)

train_data_part2 = train_preprocessor.flow_from_directory(
    train_data_path,
    target_size=(img_shape,img_shape),
    color_mode='rgb',
    batch_size=batch_size,
    shuffle = True,
    subset='training', 
    class_mode = 'categorical',
)

test_data_part2 = test_preprocessor.flow_from_directory(
    test_data_path,
    target_size=(img_shape,img_shape),
    color_mode="rgb",
    shuffle = True,
    batch_size=batch_size,
    class_mode = 'categorical',
)

#class_weights_new = compute_class_weight('balanced', np.unique(train_data_part2.classes), train_data_part2.classes)

In [ ]:
#Running the model on the defined values
base_model_5 = keras.applications.inception_v3.InceptionV3(input_shape = (80,80,3),include_top = False, weights = 'imagenet')

for i in base_model_5.layers[:-15]:
    i.trainable = False

output_model = base_model_5.output
output_model = Flatten()(output_model)
output_model = BatchNormalization()(output_model)
ouput_model = Dropout(0.2)(output_model)
output_model = Dense(512 , activation = "relu", kernel_regularizer=regularizers.l2(0.006))(output_model)
output_model = BatchNormalization()(output_model)
ouput_model = Dropout(0.2)(output_model)
output_model = Dense(128 , activation = "relu", kernel_regularizer=regularizers.l2(0.006))(output_model)
output_model = BatchNormalization()(output_model)
output_model = Dense(7 , activation  = "softmax")(output_model)

model_ite5 = Model(inputs = base_model_5.input, outputs = output_model)

model_ite5.compile(tf.keras.optimizers.Adam(), loss='CategoricalCrossentropy', metrics = ['accuracy'])

history_iter5 = model_ite5.fit(train_data_part2, epochs = 10, validation_data = test_data_part2)
plot_map(history_iter5)

According to the above one, the model got stuck => Local Minima

In [ ]:
base_model_6 = keras.applications.inception_v3.InceptionV3(input_shape = (80,80,3),include_top = False, weights = 'imagenet')

for i in base_model_6.layers[:-15]:
    i.trainable = False

output_model = base_model_6.output
output_model = Flatten()(output_model)
output_model = BatchNormalization()(output_model)
ouput_model = Dropout(0.2)(output_model)
output_model = Dense(512 , activation = "relu" )(output_model)
output_model = BatchNormalization()(output_model)
ouput_model = Dropout(0.2)(output_model)
output_model = Dense(128 , activation = "relu")(output_model)
output_model = BatchNormalization()(output_model)
output_model = Dense(7 , activation  = "softmax")(output_model)

model_ite6 = Model(inputs = base_model_6.input, outputs = output_model)

model_ite6.compile(tf.keras.optimizers.Adam(lr = 0.008), loss='CategoricalCrossentropy', metrics = ['accuracy'])

history_iter6 = model_ite6.fit(train_data_part2, epochs = 10, validation_data = test_data_part2)

#, class_weight = class_weights_new kernel_regularizer=regularizers.l2(0.008)

In [ ]:
plot_map(history_iter6)

The above graphs shows us that now we are overcoming the problem of overfitting, now we need to increase the accuracy of the model

In [ ]:
#Doing grid search on number of layers to be set to untrainable
def model_built(n_untrain):
    print("n_untrain: ", n_untrain)
    base_model_7 = keras.applications.inception_v3.InceptionV3(input_shape = (80,80,3),include_top = False, weights = 'imagenet')

    for i in base_model_7.layers[:-n_untrain]:
        i.trainable = False

    output_model = base_model_7.output
    output_model = Flatten()(output_model)
    output_model = BatchNormalization()(output_model)
    ouput_model = Dropout(0.2)(output_model)
    output_model = Dense(512 , activation = "relu" )(output_model)
    output_model = BatchNormalization()(output_model)
    ouput_model = Dropout(0.2)(output_model)
    output_model = Dense(128 , activation = "relu")(output_model)
    output_model = BatchNormalization()(output_model)
    output_model = Dense(7 , activation  = "softmax")(output_model)

    model_ite7 = Model(inputs = base_model_7.input, outputs = output_model)

    model_ite7.compile(tf.keras.optimizers.Adam(lr = 0.008), loss='SparseCategoricalCrossentropy', metrics = ['accuracy'])
    print("Model Compiled")
    return model_ite7

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adagrad, Adam

#This is the parameter values
param_grid = {
    'n_untrain': [20, 40, 25, 30],
}

#Doing grid search here and priniting out the prefered parameters
grid_search = GridSearchCV(estimator=KerasClassifier(model_built), param_grid=param_grid, scoring='accuracy', cv=3)
grid_search.fit(x_train, y_train_sparse, epochs=2)
print("Best parameters: ", grid_search.best_params_)

Our above grid search parameter shows that we should keep 40 trainable layers, so let's try it out

In [ ]:
#According to grid search we are not setting the layers to be 40 and training
base_model_7 = keras.applications.inception_v3.InceptionV3(input_shape = (80,80,3),include_top = False, weights = 'imagenet')

for i in base_model_7.layers[:-40]:
    i.trainable = False

output_model = base_model_7.output
output_model = Flatten()(output_model)
output_model = BatchNormalization()(output_model)
ouput_model = Dropout(0.2)(output_model)
output_model = Dense(512 , activation = "relu" )(output_model)
output_model = BatchNormalization()(output_model)
ouput_model = Dropout(0.2)(output_model)
output_model = Dense(128 , activation = "relu")(output_model)
output_model = BatchNormalization()(output_model)
output_model = Dense(7 , activation  = "softmax")(output_model)

model_ite7 = Model(inputs = base_model_7.input, outputs = output_model)

model_ite7.compile(tf.keras.optimizers.Adam(lr = 0.008), loss='CategoricalCrossentropy', metrics = ['accuracy'])
model_ite7.summary()
history_iter7 = model_ite7.fit(train_data_part2, epochs = 10, validation_data = test_data_part2)

#, class_weight = class_weights_new kernel_regularizer=regularizers.l2(0.008)

In [ ]:
history_iter7 = model_ite7.fit(train_data_part2, epochs = 10, validation_data = test_data_part2)
plot_map(history_iter7)

In [ ]:
base_model_7 = keras.applications.inception_v3.InceptionV3(input_shape = (80,80,3),include_top = False, weights = 'imagenet')

for i in base_model_7.layers[:-40]:
    i.trainable = False

output_model = base_model_7.output
output_model = Flatten()(output_model)
output_model = BatchNormalization()(output_model)
ouput_model = Dropout(0.2)(output_model)
output_model = Dense(512 , activation = "relu" )(output_model)
output_model = BatchNormalization()(output_model)
output_model = Dense(256 , activation = "relu")(output_model)
output_model = BatchNormalization()(output_model)
output_model = Dense(7 , activation  = "softmax")(output_model)

model_ite7 = Model(inputs = base_model_7.input, outputs = output_model)

model_ite7.compile(tf.keras.optimizers.Adam(), loss='CategoricalCrossentropy', metrics = ['accuracy'])
history_iter7 = model_ite7.fit(train_data_part2, epochs = 10, validation_data = test_data_part2)
plot_map(history_iter7)